# CSM Sandbox

## 1. Prerequisites

### Install Knoten
This tutorial requires Knoten version 0.4 or higher.

`conda install -c conda-forge knoten=0.4`

### Install Matplotlib
This tutorial requires matplotlib, ipywidgets, and ipympl.

`conda install -c conda-forge matplotlib ipywidgets ipympl`

## Imports

In [1]:
%matplotlib widget
import os                           # File Path Joining
import json                         # Read ISD as python dictionary
import copy                         # Important to be able to modify the ISD

from knoten import csm              # Knoten CSM

import matplotlib.pyplot as plt     # Math and Plotting Tools
import ipywidgets as widgets
from ipywidgets import Layout

### Methods for reading/printing stats

In [2]:
def print_stats(dict, search_keys):
    for search_key in search_keys:
        print(f"{search_key+": ":<25}" + str(dict[search_key]))

def plot_footprint(lons, lats, aspect):
    plt.close()
    plt.rcParams["figure.figsize"] = aspect
    plt.axes().set_aspect('equal','datalim')
    plt.plot(lons, lats)
    plt.xlabel("Longitude (deg)")
    plt.ylabel("Latitude (deg)")
    plt.title("CSM footprint")
    plt.show()

def plot_footprint_comparison(fp1, fp2, aspect):
    plt.clf()                                   # clear previous figure
    plt.rcParams["figure.figsize"] = aspect     # set aspect ratio of plot
    plt.axes().set_aspect('equal','datalim')    
    fp1_plot, = plt.plot(fp1[0], fp1[1], 'b')   # Plot footprint 1 in blue

    margin = 0.001                              # boundaries centered on footprint 1
    fp1_lim_x = plt.xlim()
    fp1_lim_y = plt.ylim()
    fp1_lim_x = (fp1_lim_x[0] - margin, fp1_lim_x[1] + margin)
    fp1_lim_y = (fp1_lim_y[0] - margin, fp1_lim_y[1] + margin)

    fp2_plot, = plt.plot(fp2[0], fp2[1], 'r')       # Plot footprint 2 in red
    plt.title("Original vs. Modified Footprint")    # Title and axis labels
    plt.xlabel("Longitude (deg)")
    plt.ylabel("Latitude (deg)")
    fp1_plot.set_label(fp1[2])  # Labels/Legend
    fp2_plot.set_label(fp2[2])
    plt.legend()
    plt.xlim(fp1_lim_x)         # Set Size
    plt.ylim(fp1_lim_y)
    plt.show()                  # Show plot

### Stats/Footprint of original ISD

In [3]:
# Load Dict from JSON-style ISD File
data_dir = '../data/image_to_ground'
isd_file = os.path.join(data_dir, 'isd_file.json')

with open(isd_file) as json_file:
    isd_dict = json.load(json_file)

# Print selected values from ISD
print_stats(isd_dict, ('focal_length_model', 'detector_center', 'optical_distortion', 'center_ephemeris_time'))

# Create Camera Model
camera = csm.create_csm(isd_file)

# Get the footprint using the model
boundary = csm.generate_boundary((isd_dict["image_lines"], isd_dict["image_samples"]))
lons, lats, alts = csm.generate_latlon_boundary(camera, boundary)

# # This line can plot the footprint of the original ISD
# plot_footprint(lons, lats, [5,1])

focal_length_model:      {'focal_length': 352.9271664}
detector_center:         {'line': 0.430442527, 'sample': 2542.96099}
optical_distortion:      {'radial': {'coefficients': [-0.0073433925920054505, 2.8375878636241697e-05, 1.2841989124027099e-08]}}
center_ephemeris_time:   297088762.2425226


### Modify ISD/write to file

In [ ]:
# clear the plot from any previous footprints/plots
plt.close()

# Copy the ISD Dictionary, we will modify it and compare to the original.
isd_dict_mod = copy.deepcopy(isd_dict)

print('Adjust Sliders to add or subtract from the values at the following ISD Keys:')

# Slider Widgets
wide_lay  = Layout(width='600px')
wide_desc = {'description_width': '150px'}
@widgets.interact(
        fl_add=widgets.FloatSlider(min=-0.25, max=.25, step=0.001, description='Focal Length', layout=wide_lay, style=wide_desc), 
        dcl_add=widgets.FloatSlider(min=-4, max=4, step=0.05, description='Detector Center Line', layout=wide_lay, style=wide_desc),
        dcs_add=widgets.FloatSlider(min=-2.5, max=2.5, step=0.02, description='Detector Center Sample', layout=wide_lay, style=wide_desc),
        opt_x=widgets.FloatSlider(min=-0.001, max=0.001, step=0.00001, description='Optical Distortion X', layout=wide_lay, style=wide_desc, readout_format='.5f'),
        opt_y=widgets.FloatSlider(min=-2e-6, max=2e-6, step=2e-8, description='Optical Distortion Y', layout=wide_lay, style=wide_desc, readout_format='.1e'),
        opt_z=widgets.FloatSlider(min=-1e-8, max=1e-8, step=1e-10, description='Optical Distortion Z', layout=wide_lay, style=wide_desc, readout_format='.1e'),
        ect_add=widgets.FloatSlider(min=-1e-2, max=1e-2, step=1e-4, description='Exposure (Center) Time', layout=wide_lay, style=wide_desc, readout_format='.4f')
    )
# This function executed whenever one of the slider widgets is adjusted
def exec_widget_function(fl_add, dcl_add, dcs_add, opt_x, opt_y, opt_z, ect_add):

    # If you're curious where the ISD values came from, 
    # Detector Center was from NAIF Boresight Line/Sample
    # Optical Distortion was from NAIF OD_K
    # ISIS uses the NAIF Keywords, but Knoten CSM uses other derived ISD values.

    new_values = {
        'focal_length_model': {'focal_length': 352.9271664 + fl_add},
        'detector_center':    {'line': 0.430442527 + dcl_add, 'sample': 2542.96099 + dcs_add},
        'optical_distortion': {'radial': {'coefficients': [-0.007343 + opt_x, 2.838e-05 + opt_y, 1.284e-08 + opt_z]}},
        'center_ephemeris_time': 297088762.2425226 + ect_add
    }

    # Modify Values in Dictionary
    for key,value in new_values.items(): 
        isd_dict_mod[key] = new_values[key]

    # Write ISD to file
    isd_file_mod = os.path.join(data_dir, 'isd_file_mod.json')
    with open(isd_file_mod, 'w') as json_file:
        json.dump(isd_dict_mod, json_file, indent=4)

    print_stats(isd_dict_mod, ('focal_length_model', 'detector_center', 'optical_distortion', 'center_ephemeris_time'))

    # Create Camera Model
    camera = csm.create_csm(isd_file_mod)

    # Get the footprint using the model
    boundary_mod = csm.generate_boundary((isd_dict_mod["image_lines"], isd_dict_mod["image_samples"]))
    lons_mod, lats_mod, alts_mod = csm.generate_latlon_boundary(camera, boundary_mod)

    # Plot it
    plot_footprint_comparison((lons, lats, "Original"),(lons_mod, lats_mod, "Modified"), [7,3])

Adjust Sliders to add or subtract from the values at the following ISD Keys:


interactive(children=(FloatSlider(value=0.0, description='Focal Length', layout=Layout(width='600px'), max=0.2…